# Model Optimization

This notebook focuses on making models lightweight and efficient using:
1. Model pruning
2. Weight quantization
3. Tensor compression
4. Architecture optimization

In [ ]:
import os
import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# Create models directory
os.makedirs('models', exist_ok=True)

## 1. Load Training Results

In [ ]:
# Load training results
with open('results/training_results.pkl', 'rb') as f:
    results = pickle.load(f)

# Compare model sizes and accuracies
print("Model Performance Summary:")
print("-" * 50)
for name, result in results.items():
    model = result['model']
    params = model.count_params()
    accuracy = result['accuracy']
    print(f"{name}:")
    print(f"Parameters: {params:,}")
    print(f"Accuracy: {accuracy:.4f}\n")

## 2. Model Size Analysis

In [ ]:
def analyze_model_size(model, name):
    """Analyze model size and complexity"""
    # Save original model
    base_path = f'models/{name}'
    model.save(f'{base_path}_original.h5')
    original_size = os.path.getsize(f'{base_path}_original.h5') / 1024  # KB
    
    # Get parameter count by layer
    layer_params = [(layer.name, layer.count_params()) for layer in model.layers]
    
    print(f"\nModel: {name}")
    print(f"Total size: {original_size:.2f} KB")
    print("\nParameters by layer:")
    for layer_name, params in layer_params:
        print(f"{layer_name}: {params:,}")
    
    return original_size, layer_params

# Analyze all models
size_analysis = {}
for name, result in results.items():
    size_analysis[name] = analyze_model_size(result['model'], name)

# Plot model sizes
plt.figure(figsize=(10, 6))
names = list(size_analysis.keys())
sizes = [analysis[0] for analysis in size_analysis.values()]

plt.bar(names, sizes)
plt.title('Model Size Comparison')
plt.ylabel('Size (KB)')
plt.xticks(rotation=45)

for i, v in enumerate(sizes):
    plt.text(i, v, f'{v:.1f}KB', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 3. Model Optimization Functions

In [ ]:
def quantize_model(model):
    """Quantize model to reduce size"""
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_types = [tf.float16]
    converter.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS,
        tf.lite.OpsSet.SELECT_TF_OPS
    ]
    return converter.convert()

def compress_weights(model):
    """Apply weight compression techniques"""
    for layer in model.layers:
        if hasattr(layer, 'kernel'):
            # Apply 16-bit floating point
            weights = layer.get_weights()
            weights = [w.astype(np.float16) for w in weights]
            layer.set_weights(weights)
    return model

def optimize_architecture(model):
    """Optimize model architecture"""
    # Remove unnecessary layers
    optimized_model = tf.keras.models.Sequential()
    
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Dropout):
            # Adjust dropout rate
            optimized_model.add(tf.keras.layers.Dropout(0.1))
        elif isinstance(layer, tf.keras.layers.Dense):
            # Reduce dense layer size
            units = layer.units
            if layer != model.layers[-1]:  # Don't modify output layer
                units = units // 2
            optimized_model.add(tf.keras.layers.Dense(units, activation=layer.activation))
        else:
            optimized_model.add(layer)
    
    return optimized_model

## 4. Apply Optimizations

In [ ]:
def optimize_model(model, name, X_test, y_test):
    """Apply all optimization techniques and evaluate results"""
    print(f"\nOptimizing {name}...")
    
    # Save original metrics
    original_size = os.path.getsize(f'models/{name}_original.h5') / 1024
    y_pred = np.argmax(model.predict(X_test), axis=1)
    original_accuracy = accuracy_score(y_test, y_pred)
    
    results = {
        'Original': {
            'size': original_size,
            'accuracy': original_accuracy
        }
    }
    
    try:
        # 1. Architecture optimization
        arch_optimized = optimize_architecture(model)
        arch_optimized.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        arch_optimized.fit(X_test, y_test, epochs=5, verbose=0)  # Quick fine-tuning
        
        arch_optimized.save(f'models/{name}_arch_opt.h5')
        y_pred = np.argmax(arch_optimized.predict(X_test), axis=1)
        
        results['Architecture Optimized'] = {
            'size': os.path.getsize(f'models/{name}_arch_opt.h5') / 1024,
            'accuracy': accuracy_score(y_test, y_pred)
        }
        
        # 2. Weight compression
        compressed = compress_weights(arch_optimized)
        compressed.save(f'models/{name}_compressed.h5')
        y_pred = np.argmax(compressed.predict(X_test), axis=1)
        
        results['Compressed'] = {
            'size': os.path.getsize(f'models/{name}_compressed.h5') / 1024,
            'accuracy': accuracy_score(y_test, y_pred)
        }
        
        # 3. Quantization
        quantized = quantize_model(compressed)
        with open(f'models/{name}_quantized.tflite', 'wb') as f:
            f.write(quantized)
            
        results['Quantized'] = {
            'size': os.path.getsize(f'models/{name}_quantized.tflite') / 1024,
            'accuracy': results['Compressed']['accuracy']  # Same as compressed model
        }
        
        print(f"\nOptimization Results for {name}:")
        for version, metrics in results.items():
            print(f"\n{version}:")
            print(f"Size: {metrics['size']:.2f} KB")
            print(f"Accuracy: {metrics['accuracy']:.4f}")
        
        return True, results
        
    except Exception as e:
        print(f"\nError during optimization: {str(e)}")
        return False, results

# Load test data
X_test = np.load('preprocessed/X_test.npy')
y_test = np.load('preprocessed/y_test.npy')
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Optimize all models
optimization_results = {}
for name, result in results.items():
    success, metrics = optimize_model(result['model'], name, X_test_reshaped, y_test)
    if success:
        optimization_results[name] = metrics

## 5. Visualization of Results

In [ ]:
def plot_optimization_results(results):
    """Plot size and accuracy comparisons"""
    plt.figure(figsize=(15, 5))
    
    # Size comparison
    plt.subplot(1, 2, 1)
    for name, metrics in results.items():
        versions = list(metrics.keys())
        sizes = [metrics[v]['size'] for v in versions]
        x = range(len(versions))
        plt.plot(x, sizes, marker='o', label=name)
    
    plt.title('Model Size Comparison')
    plt.xlabel('Optimization Stage')
    plt.ylabel('Size (KB)')
    plt.xticks(range(len(versions)), versions, rotation=45)
    plt.legend()
    
    # Accuracy comparison
    plt.subplot(1, 2, 2)
    for name, metrics in results.items():
        versions = list(metrics.keys())
        accuracies = [metrics[v]['accuracy'] for v in versions]
        x = range(len(versions))
        plt.plot(x, accuracies, marker='o', label=name)
    
    plt.title('Model Accuracy Comparison')
    plt.xlabel('Optimization Stage')
    plt.ylabel('Accuracy')
    plt.xticks(range(len(versions)), versions, rotation=45)
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Plot results
plot_optimization_results(optimization_results)

# Save optimization results
with open('results/optimization_results.pkl', 'wb') as f:
    pickle.dump(optimization_results, f)

print("\nOptimization results saved successfully!")